Import some libraries we'll use:

In [8]:
import urllib.request
import gzip
import pandas as pd
import os

Download the data from NOAA. The columns in the file are documented [here](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/readme.txt).

In [22]:
file = '1975.csv.gz'
if os.path.isfile(file):
    print(file, 'already downloaded.')
else:
    urllib.request.urlretrieve('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/1975.csv.gz', file)

1975.csv.gz already downloaded.


Before we can read the file, we'll define our own date parser:

In [23]:
noaadateparser = lambda x: pd.datetime.strptime(x, "%Y%m%d")


Unzip the file and load the CSV with pandas, using our date parser. We'll also give the columns names since the CSV doesn't have a header 🧟‍:

In [24]:
with gzip.open('1975.csv.gz') as f:

    data_1975 = pd.read_csv(f, 
                            names = ["station", "date", "type", "value"],
                            usecols = [0,1,2,3],
                            parse_dates = ["date"],
                            date_parser = noaadateparser)

data_1975.head()

,station,date,type,value
0,CA008203164,1975-01-01,TMAX,17
1,CA008203164,1975-01-01,TMIN,-11
2,CA008203164,1975-01-01,PRCP,216
3,CA008203164,1975-01-01,SNOW,216
4,CA008105560,1975-01-01,TMAX,17


In [25]:
data_1975.dtypes

station            object
date       datetime64[ns]
type               object
value               int64
dtype: object

Only keep TMIN and TMAX:

In [26]:
data_1975 = data_1975[(data_1975["type"] == "TMAX") | (data_1975["type"] == "TMIN")] 

In [27]:
data_1975

,station,date,type,value
0,CA008203164,1975-01-01,TMAX,17
1,CA008203164,1975-01-01,TMIN,-11
4,CA008105560,1975-01-01,TMAX,17
5,CA008105560,1975-01-01,TMIN,-67
10,CA007056600,1975-01-01,TMAX,-22
11,CA007056600,1975-01-01,TMIN,-111
15,CA007048421,1975-01-01,TMAX,-100
16,CA007048421,1975-01-01,TMIN,-278
19,CA007045400,1975-01-01,TMAX,-139
20,CA007045400,1975-01-01,TMIN,-261


Next, let's get the list of stations so that we have lat/lon for each:

In [28]:
stations = pd.read_fwf("https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt", 
            infer_nrows=300, # how many rows to use to infer the column widths
            usecols = [0,1,2,3,4],
            names = ["station", "lat", "lon", "elevation", "name"])

stations.head()

,station,lat,lon,elevation,name
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL


Join the two together:

In [29]:
data_1975 = pd.merge(data_1975, stations, on="station")
data_1975.head()

,station,date,type,value,lat,lon,elevation,name
0,CA008203164,1975-01-01,TMAX,17,45.6,-60.75,15.0,LOWER L'ARDOISE
1,CA008203164,1975-01-01,TMIN,-11,45.6,-60.75,15.0,LOWER L'ARDOISE
2,CA008203164,1975-01-02,TMAX,-11,45.6,-60.75,15.0,LOWER L'ARDOISE
3,CA008203164,1975-01-02,TMIN,-72,45.6,-60.75,15.0,LOWER L'ARDOISE
4,CA008203164,1975-01-03,TMAX,-11,45.6,-60.75,15.0,LOWER L'ARDOISE


Pull out the country ID from the station column (first two letters):

In [30]:
data_1975["country"] = data_1975["station"].astype(str).str[0:2]
data_1975

,station,date,type,value,lat,lon,elevation,name,country
0,CA008203164,1975-01-01,TMAX,17,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
1,CA008203164,1975-01-01,TMIN,-11,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
2,CA008203164,1975-01-02,TMAX,-11,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
3,CA008203164,1975-01-02,TMIN,-72,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
4,CA008203164,1975-01-03,TMAX,-11,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
5,CA008203164,1975-01-03,TMIN,-72,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
6,CA008203164,1975-01-04,TMAX,33,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
7,CA008203164,1975-01-04,TMIN,-61,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
8,CA008203164,1975-01-05,TMAX,-6,45.600,-60.750,15.0,LOWER L'ARDOISE,CA
9,CA008203164,1975-01-05,TMIN,-33,45.600,-60.750,15.0,LOWER L'ARDOISE,CA


Index the dataframe by station, date and observation type:

In [31]:
data_1975.set_index(['station','date','type'], inplace=True)
data_1975

value     lat     lon  elevation  \
station     date       type                                     
CA008203164 1975-01-01 TMAX     17  45.600 -60.750       15.0   
                       TMIN    -11  45.600 -60.750       15.0   
            1975-01-02 TMAX    -11  45.600 -60.750       15.0   
                       TMIN    -72  45.600 -60.750       15.0   
            1975-01-03 TMAX    -11  45.600 -60.750       15.0   
                       TMIN    -72  45.600 -60.750       15.0   
            1975-01-04 TMAX     33  45.600 -60.750       15.0   
                       TMIN    -61  45.600 -60.750       15.0   
            1975-01-05 TMAX     -6  45.600 -60.750       15.0   
                       TMIN    -33  45.600 -60.750       15.0   
            1975-01-06 TMAX    -17  45.600 -60.750       15.0   
                       TMIN    -67  45.600 -60.750       15.0   
            1975-01-07 TMAX     17  45.600 -60.750       15.0   
                       TMIN    -94  45.600 -60.750       15.0   
            1975-01-08 TMAX     22  45.600 -60.750       15.0   
                       TMIN    -11  45.600 -60.750       15.0   
            1975-01-09 TMAX     11  45.600 -60.750       15.0   
                       TMIN    -28  45.600 -60.750       15.0   
            1975-01-10 TMAX     22  45.600 -60.750       15.0   
                       TMIN    -28  45.600 -60.750       15.0   
            1975-01-11 TMAX     44  45.600 -60.750       15.0   
                       TMIN      0  45.600 -60.750       15.0   
            1975-01-12 TMAX     78  45.600 -60.750       15.0   
                       TMIN      6  45.600 -60.750       15.0   
            1975-01-13 TMAX     72  45.600 -60.750       15.0   
                       TMIN      0  45.600 -60.750       15.0   
            1975-01-14 TMAX     17  45.600 -60.750       15.0   
                       TMIN    -11  45.600 -60.750       15.0   
            1975-01-15 TMAX      6  45.600 -60.750       15.0   
                       TMIN    -56  45.600 -60.750       15.0   
...                            ...     ...     ...        ...   
MXN00031055 1975-12-22 TMAX    278  21.000 -88.300       26.8   
                       TMIN     56  21.000 -88.300       26.8   
            1975-12-23 TMAX    317  21.000 -88.300       26.8   
                       TMIN     56  21.000 -88.300       26.8   
            1975-12-24 TMAX    328  21.000 -88.300       26.8   
                       TMIN    117  21.000 -88.300       26.8   
            1975-12-25 TMAX    317  21.000 -88.300       26.8   
                       TMIN    178  21.000 -88.300       26.8   
            1975-12-26 TMAX    267  21.000 -88.300       26.8   
                       TMIN    189  21.000 -88.300       26.8   
            1975-12-27 TMAX    317  21.000 -88.300       26.8   
                       TMIN    150  21.000 -88.300       26.8   
            1975-12-28 TMAX    328  21.000 -88.300       26.8   
                       TMIN    167  21.000 -88.300       26.8   
            1975-12-29 TMAX    339  21.000 -88.300       26.8   
                       TMIN    117  21.000 -88.300       26.8   
            1975-12-30 TMAX    339  21.000 -88.300       26.8   
                       TMIN    106  21.000 -88.300       26.8   
            1975-12-31 TMAX    339  21.000 -88.300       26.8   
                       TMIN    156  21.000 -88.300       26.8   
RSM00022727 1975-12-13 TMIN    -20  62.167  34.300       38.0   
            1975-12-19 TMIN   -150  62.167  34.300       38.0   
            1975-12-21 TMIN   -110  62.167  34.300       38.0   
RSM00022833 1975-12-22 TMIN    -60  61.783  37.750      116.0   
RSM00022915 1975-12-22 TMIN    -10  60.467  32.900        8.0   
            1975-12-25 TMIN   -110  60.467  32.900        8.0   
MXM00076741 1975-12-28 TMAX    220  18.133 -94.417       22.0   
                       TMIN    190  18.133 -94.417       22.0   
MXM00076737 1975-12-29 TMIN    110  18.850 -97.100     1259.0   